In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
from scipy.io import loadmat
from sklearn.svm import SVC

In [2]:
def preprocess():
    f = open("abalone.txt", 'r')
    o = open("abaloneProcessed.txt", 'w')
    c = f.read(1)
    while True:
        if not c:
            break
        if c == 'M':
            o.write('1')
        elif c == 'F':
            o.write('-1')
        elif c == 'I':
            o.write('0')
        else:
            o.write(c)
        c = f.read(1)
    f.close()
    o.close()

# SVM

In [3]:
def correctPercentage(svm, x, y, rango):
    array = svm.predict(x) #Guardamos los valores de la función sigmoide en el array 
    #aprobados = (array >= 0.5) #Guardamos en un array los aprobados("true") y suspensos("false")
    cmp = (np.abs(array - y) <= rango) #Comparamos con los resultados originales y guardamos las coincidencias("true")
    return np.sum(cmp)/len(y)*100 #Devolvemos el porcentaje de acierto

In [4]:
def loadData():
    data = np.genfromtxt('abaloneProcessed.txt', delimiter=',')
    
    x = data[:,:-1]
    y = data[:,-1]
    
    join = np.hstack((x, np.c_[y]))
    np.random.shuffle(join)
    
    #Se vuelven a separar tras barajearse
    xShuffled = join[:,:-1]
    yShuffled = join[:,-1]
    
    xTraining = xShuffled[:2700]
    yTraining = yShuffled[:2700]
    xVal = xShuffled[2700:3800]
    yVal = yShuffled[2700:3800]
    xTest = xShuffled[3800:]
    yTest = yShuffled[3800:]
    
    return xTraining, yTraining, xVal, yVal, xTest, yTest

In [5]:
def SVM(rango): 
    
    xTraining, yTraining, xVal, yVal, xTest, yTest = loadData()
    bestvalue = -1
    nums = np.array([0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30])
    
    for sigma in nums:
        for c in nums:
            svm = SVC(kernel='rbf', C=c ,gamma=1 / (2 * sigma**2))
            svm.fit(xTraining,yTraining)
            per = correctPercentage(svm,xVal,yVal, rango)
            perEnt = correctPercentage(svm,xTraining,yTraining, rango)
            if per >= bestvalue:
                bestc = c
                bestsigma = sigma
                bestvalue = per
                bestsvm = svm
            print("%.2f para C = %.2f y sigma = %.2f" % (per, c, sigma))
            print("#%.2f para C = %.2f y sigma = %.2f (entrenamiento)" % (perEnt, c, sigma))
    
    print("Mejor porcentaje de acierto sobre los ejemplos de validación: %.2f" % bestvalue)
    #y = y + 1.5
    svm = SVC(kernel='rbf', C=30 ,gamma=1 / (2 * 0.01**2))
    svm.fit(xTraining,yTraining)
    print(svm.predict(xVal))

In [6]:
SVM(2.)

65.36 para C = 0.01 y sigma = 0.01
#66.81 para C = 0.01 y sigma = 0.01 (entrenamiento)
65.36 para C = 0.03 y sigma = 0.01
#66.81 para C = 0.03 y sigma = 0.01 (entrenamiento)
65.36 para C = 0.10 y sigma = 0.01
#66.81 para C = 0.10 y sigma = 0.01 (entrenamiento)
65.36 para C = 0.30 y sigma = 0.01
#66.81 para C = 0.30 y sigma = 0.01 (entrenamiento)
70.73 para C = 1.00 y sigma = 0.01
#99.78 para C = 1.00 y sigma = 0.01 (entrenamiento)
70.91 para C = 3.00 y sigma = 0.01
#100.00 para C = 3.00 y sigma = 0.01 (entrenamiento)
70.91 para C = 10.00 y sigma = 0.01
#100.00 para C = 10.00 y sigma = 0.01 (entrenamiento)
70.91 para C = 30.00 y sigma = 0.01
#100.00 para C = 30.00 y sigma = 0.01 (entrenamiento)
65.36 para C = 0.01 y sigma = 0.03
#66.81 para C = 0.01 y sigma = 0.03 (entrenamiento)
65.36 para C = 0.03 y sigma = 0.03
#66.81 para C = 0.03 y sigma = 0.03 (entrenamiento)
69.91 para C = 0.10 y sigma = 0.03
#70.93 para C = 0.10 y sigma = 0.03 (entrenamiento)
73.73 para C = 0.30 y sigma = 0.03
#

# Neural network

In [7]:
def sigmoide(x):
    return 1./(1. + np.exp(-1. * x))

In [8]:
def pesosAleatorios(Lin, Lout):
    ini = 0.12 
    theta = np.random.random((Lout,Lin+1))*(2*ini)-ini
    return theta

In [9]:
def h0(x,theta1,theta2):
    z2 = np.matmul(x, theta1.T)
    a2 = sigmoide(z2)
    a2 = np.hstack((np.ones((a2.shape[0],1)), a2))
    
    #Capa de salida
    z3 = np.matmul(theta2, a2.T)
    a3 = sigmoide(z3)
    
    return a3, a2 

In [35]:
def coste(xOnes, y2, theta1, theta2, reg):
    a3, a2 = h0(xOnes, theta1, theta2)
    a = - y2 * np.log(a3).T
    b = (1 - y2) * np.log(1 - a3).T
    c = np.sum(1. / (xOnes.shape[0]) * (a - b))
    coste = c + (reg/(2 * xOnes.shape[0]))*(np.sum(theta1**2) + np.sum(theta2**2))
    return coste

In [36]:
#Solo funciona para 1 capa oculta
def backdrop(params_rn, num_entradas, num_ocultas, num_etiquetas, x, y, reg = 0):
    #Desempaqueta los parámetros
    theta1 = np.reshape(params_rn[: num_ocultas * (num_entradas + 1) ],
                        (num_ocultas , (num_entradas + 1)))
    theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1) :],
                        (num_etiquetas , (num_ocultas + 1)))
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))

    #Genera la matriz de etiquetas.
    y2 = np.zeros((len(y), num_etiquetas))
    miniy = int(np.min(y))
    #Pone los 1 en la matriz de 0
    for i in range(len(y)):
        y2[i,int(y[i]) - miniy] = 1
    
    #FORWARD PROPAGATION-------------------------------------------------
    a3, a2 = h0(xOnes, theta1, theta2)
    a = - y2 * np.log(a3).T
    b = (1 - y2) * np.log(1 - a3).T
    c = np.sum(1. / (xOnes.shape[0]) * (a - b))
    coste = c + (reg/(2 * xOnes.shape[0]))*(np.sum(theta1**2) + np.sum(theta2**2))

    #BACKPROPAGATION------------------------------------------------------
    delta3 = a3 - y2.T
    delta2 = np.matmul(theta2.T, delta3) * (a2.T * (1. - a2.T))
    #Copia theta para evitar dañar la original para vectorizar que
    #la primera fila no se modifica al regularizar
    theta1c = np.copy(theta1)
    theta2c = np.copy(theta2)
    theta2c[:,0] = theta2c[:,0] * 0  
    theta1c[:,0] = theta1c[:,0] * 0
    
    #Calcula las theta resultantes, incluyendo la regularización
    triangulo2 = (np.matmul(delta3, a2) / xOnes.shape[0]) 
    triangulo2 = triangulo2 + (reg/xOnes.shape[0]) * theta2c
    triangulo1 = (np.matmul(delta2[1:], xOnes) / xOnes.shape[0]) 
    triangulo1 = triangulo1 + (reg/xOnes.shape[0]) * theta1c
    
    return coste, np.concatenate((triangulo1.ravel(),triangulo2.ravel()))

In [37]:
def checkNeural(theta_opt1, theta_opt2, x, y, maxi, mini):
    xOnes = np.hstack((np.ones((x.shape[0],1)), x))
    for i in range(xOnes.shape[0]):
        salida = h0(xOnes[i,np.newaxis], theta_opt1, theta_opt2)
        salida = salida[0].ravel()
        print((salida.argmax() + mini) % maxi, "con un valor h0 de",  salida.max())
        print("%i es el valor real" % int(y[i]))


In [38]:
def neuralNetwork(reg):
    xTraining, yTraining, xVal, yVal, xTest, yTest = loadData()
    num_entradas = xTraining.shape[1]
    num_ocultas = 25
    #print(np.max(yTraining), np.min(yTraining))
    
    #Utiliza el mínimo y máximo para crear el número de etiquetas,
    #Puede variar entre ejecuciones al elegir ejemplos aleatoriamente,
    #Ya que tienen una distribució normal
    maxi = int(np.max(yTraining))
    mini = int(np.min(yTraining))
    num_etiquetas = maxi - mini + 1
    
    #print(num_etiquetas)
    
    t1 = pesosAleatorios(num_entradas,num_ocultas)
    t2 = pesosAleatorios(num_ocultas, num_etiquetas)
    params_t = np.vstack((np.reshape(t1, (t1.shape[0]*t1.shape[1],1)),
                           np.reshape(t2, (t2.shape[0]*t2.shape[1],1))))
    
    #Entrena la red neuronal
    result = opt.minimize(fun=backdrop, x0=params_t,
                          args=(num_entradas, num_ocultas,
                                num_etiquetas, xTraining, yTraining, reg),
                          method='TNC', jac=True)
    
    #Desempaqueta la salida del entrenamiento
    salida1 = np.reshape(result.x[: num_ocultas * (num_entradas + 1) ], 
                         (num_ocultas , (num_entradas + 1)))
    salida2 = np.reshape(result.x[num_ocultas * (num_entradas + 1) :], 
                         (num_etiquetas , (num_ocultas + 1)))
    
    checkNeural(salida1, salida2, xVal[0:20], yVal[0:20], maxi, mini)
    
    return salida1, salida2, result.fun, maxi, mini



In [39]:
def neuralNetworkDecide():
    xTraining, yTraining, xVal, yVal, xTest, yTest = loadData()
    num_entradas = xTraining.shape[1]
    num_ocultas = 25
    #print(np.max(yTraining), np.min(yTraining))
    
    #Utiliza el mínimo y máximo para crear el número de etiquetas,
    #Se utiliza el minimo y máximo entre todos los ejemplos, aunque
    #no aparezcan en los de entranamineto
    maxi = int(np.max([np.max(yTraining), np.max(yVal), np.max(yTest)]))
    mini = int(np.min([np.min(yTraining), np.min(yVal), np.min(yTest)]))
    print(maxi,mini)
    num_etiquetas = maxi - mini + 1
    
    #print(num_etiquetas)
    
    xValOnes = np.hstack((np.ones((xVal.shape[0],1)), xVal))

    #Genera la matriz de etiquetas.
    yVal2 = np.zeros((len(yVal), num_etiquetas))
    #Pone los 1 en la matriz de 0
    for i in range(len(yVal)):
        yVal2[i,int(yVal[i]) - mini] = 1
    
    reg = np.array([0.001, 0.003, 0.007, 0.01, 0.03, 0.07, 
                    0.1, 0.3, 0.7, 1, 3, 7, 10, 30])
    #Entrena la red neuronal
    for i in reg:
        t1 = pesosAleatorios(num_entradas,num_ocultas)
        t2 = pesosAleatorios(num_ocultas, num_etiquetas)
        params_t = np.vstack((np.reshape(t1, (t1.shape[0]*t1.shape[1],1)),
                            np.reshape(t2, (t2.shape[0]*t2.shape[1],1))))
        result = opt.minimize(fun=backdrop, x0=params_t,
                              args=(num_entradas, num_ocultas,
                                    num_etiquetas, xTraining, yTraining, i),
                              method='TNC', jac=True)
    
    #Desempaqueta la salida del entrenamiento
        salida1 = np.reshape(result.x[: num_ocultas * (num_entradas + 1) ], 
                             (num_ocultas , (num_entradas + 1)))
        salida2 = np.reshape(result.x[num_ocultas * (num_entradas + 1) :], 
                             (num_etiquetas , (num_ocultas + 1)))
        
        cost = coste(xValOnes, yVal2, salida1, salida2, i)
        print("%.5f para reg = %.3f" % (cost, i))
    
    checkNeural(salida1, salida2, xVal[0:20], yVal[0:20], maxi, mini)
    
    return salida1, salida2, result.fun, maxi, mini


In [40]:
theta_opt1, theta_opt2, coste_opt, maxi, mini = neuralNetwork(0)
print("COSTE", coste_opt)

C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply


9 con un valor h0 de 0.2274800150759529
11 es el valor real
9 con un valor h0 de 0.22051240617581153
9 es el valor real
8 con un valor h0 de 0.3180418290234178
10 es el valor real
8 con un valor h0 de 0.3111725401297824
8 es el valor real
10 con un valor h0 de 0.324781858972899
10 es el valor real
10 con un valor h0 de 0.2477256272212695
9 es el valor real
10 con un valor h0 de 0.3259515734014004
14 es el valor real
10 con un valor h0 de 0.3091391604289531
10 es el valor real
11 con un valor h0 de 0.4004792007798561
12 es el valor real
7 con un valor h0 de 0.4822776145967882
7 es el valor real
13 con un valor h0 de 0.15666870588447213
19 es el valor real
9 con un valor h0 de 0.3125544370853389
8 es el valor real
7 con un valor h0 de 0.44461343834650674
8 es el valor real
10 con un valor h0 de 0.16600737314360905
9 es el valor real
9 con un valor h0 de 0.28511334470408567
7 es el valor real
13 con un valor h0 de 0.38654181294578643
21 es el valor real
9 con un valor h0 de 0.381195809638

In [41]:
neuralNetworkDecide()

29 1


C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
C:\Users\Miguel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply


3.33351 para reg = 0.001
3.29173 para reg = 0.003
3.63243 para reg = 0.007
3.35247 para reg = 0.010
3.44786 para reg = 0.030
3.39373 para reg = 0.070
3.40293 para reg = 0.100
3.45199 para reg = 0.300
3.55736 para reg = 0.700
3.57885 para reg = 1.000
3.71293 para reg = 3.000
4.00940 para reg = 7.000
4.09149 para reg = 10.000
4.30155 para reg = 30.000
8 con un valor h0 de 0.19585425775863133
5 es el valor real
8 con un valor h0 de 0.190471916986668
8 es el valor real
8 con un valor h0 de 0.18821883643389786
8 es el valor real
8 con un valor h0 de 0.18286584031703096
14 es el valor real
8 con un valor h0 de 0.1907230856600843
6 es el valor real
8 con un valor h0 de 0.17783703903692513
20 es el valor real
8 con un valor h0 de 0.17781926321650035
13 es el valor real
8 con un valor h0 de 0.18406288353531125
8 es el valor real
8 con un valor h0 de 0.17725006167473883
9 es el valor real
8 con un valor h0 de 0.18530418759580777
8 es el valor real
8 con un valor h0 de 0.1709574698780437
14 es el

(array([[ 0.59843552, -0.01788844,  0.24247536,  0.1053231 ,  0.03380003,
          0.16193686,  0.13598096,  0.00676424,  0.05815886],
        [ 0.5577679 ,  0.05370242,  0.20714091,  0.15268207,  0.03904425,
          0.32802722,  0.20172468,  0.09618347,  0.12377531],
        [ 0.5332895 ,  0.03748576,  0.14951144,  0.19983278,  0.00525763,
          0.33082754,  0.15249791,  0.05896006,  0.07854654],
        [ 0.53411759,  0.05201045,  0.20644912,  0.13989592,  0.04290564,
          0.43472133,  0.13297316,  0.03414679,  0.15547437],
        [ 0.52783991, -0.03766261,  0.20991159,  0.1652543 ,  0.07363646,
          0.23265456,  0.11983924,  0.0322619 ,  0.07455964],
        [ 0.68416079,  0.05708981,  0.20325967,  0.13128917,  0.00952217,
          0.33188429,  0.07222332,  0.00791247,  0.11845834],
        [ 0.77102425,  0.066856  ,  0.1859344 ,  0.11197695,  0.04035744,
          0.22758306,  0.05475554,  0.03423485,  0.0574373 ],
        [ 0.63388654,  0.01295431,  0.20844049, 